In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import re
import utils
from bs4 import BeautifulSoup

import nltk

In [2]:
products = pd.read_csv('data/DreamMarket_master_product.csv')

C:\Users\morga\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#utils.plot_null_val_heatmap(products_v2, 'Null Values of Raw "Products" Dataset')

## Reference Objects

In [4]:
country_map = {'united states': 'usa', 'united kingdom' :'uk', 'dutch': 'netherlands', 'ww': 'worldwide', 'holland': 'netherlands', 'amsterdam': 'netherlands', 'nederland': 'netherlands',
              'españa': 'spain', 'au': 'australia', 'deutschland': 'germany'}

## Functions - Add to utils.py

## Review Raw Data

In [5]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130936 entries, 0 to 130935
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   seqNo             39473 non-null   float64
 1   product_name      130936 non-null  object 
 2   category          130936 non-null  object 
 3   description       130936 non-null  object 
 4   shipping_options  130936 non-null  object 
 5   keywords          130936 non-null  object 
 6   seller_name       130936 non-null  object 
 7   price             130936 non-null  object 
 8   payment_method    130936 non-null  object 
 9   sold_since        130936 non-null  object 
 10  ends_in           130936 non-null  object 
 11  quantity_sold     130936 non-null  object 
 12  quantity_left     130936 non-null  object 
 13  refundPolicy      130936 non-null  object 
 14  market_name       130936 non-null  object 
 15  ship_from         130932 non-null  object 
 16  ship_to           13

In [6]:
cols_to_review = ['sold_since', 'ends_in', 'quantity_sold', 'quantity_left', 'refundPolicy', 'keywords', 'market_name']
for x in cols_to_review:
    print('{}: {}'.format(x, products[x].unique()))

sold_since: ['-']
ends_in: ['-']
quantity_sold: ['-']
quantity_left: ['-']
refundPolicy: ['-']
keywords: ['-']
market_name: ['Dream']


In [7]:
def select_non_null_value(x, y):
    '''
    Returns the non-null value. 
    If neither value is null, returns x.
    
    Args:
        x (float) : Float or
        y (float) : number of integer in the product id excluding the year.
         
    Return:
        res : Non-null value
    '''
    
    if not np.isnan(x):
        res = x
    else:
        res = y
    return res

def create_product_id(seq, num_ints, year):
    '''
    Creates a product id by merging the initial sequence number or product id with the year the product was offered.
    Adds zeros to the front of the "seq" value to ensure number of integers.
    
    Args:
        seq (int) : A sequence of numbers.
        num_int (int) : number of integer in the product id excluding the year.
        year (int) : Four digits.
        
    Return:
        product_id (str) : Identifier for the product. Format is "yy-nnnnnn". Number of digits ("n") can vary.
    
    '''
    
    seq_str = str(int(seq)).zfill(num_ints)
    year_str = str(year)[2:]
    product_id = year_str + '-' + seq_str
    
#     try:
        
#     except:
#         product_id = np.nan
        
         
    return product_id

## Text Cleaning

In [8]:
products_v2 = products.drop(cols_to_review, axis=1)

In [9]:
# Create sale year of posting
products_v2['is_2016'] = products_v2['idproduct'].isnull()
products_v2['year_posted'] = products_v2['is_2016'].apply(lambda x: 2016 if True else 2017)
products_v2.drop('is_2016', axis=1, inplace=True)

# Convert listing price from string to float.
products_v2['adj_price'] = products_v2['price'].apply(lambda x: utils.convert_price_to_float(x))

In [10]:
products_v2.head()

,seqNo,product_name,category,description,shipping_options,seller_name,price,payment_method,ship_from,ship_to,idproduct,year_posted,adj_price
0,1.0,503 - Forwarding failure (Privoxy@localhost),-,-,-,-,-,-,-,-,NaN,2016,NaN
1,2.0,OG KUSH 1/2 OZ $125 AAAA+,Drugs \n\t\t41793,OG Kush makes up the genetic backbone of West ...,\n\n\n$15.99\t\t\t\t\t\nCANADA WIDE\n\n\n\n\n\...,GWLOVE,$125,Escrow Yes,CANADA,Worldwide Worldwide,NaN,2016,125.0
2,3.0,Dream Market Login - Featured anonymous market...,-,-,-,-,-,-,-,-,NaN,2016,NaN
3,4.0,Every drug available on request,Drugs \n\t\t41804,Every kind of medicine I can arrange . Send me...,-,Hectorstuff,$1,Escrow Yes,dutch,Worldwide Worldwide,NaN,2016,1.0
4,5.0,"Debug, Learn how crack windows",Digital Goods \n\t\t29334,"Debug, Learn how crack windows",-,darkmarket,$1,Escrow Yes,Worldwide,Worldwide Worldwide,NaN,2016,1.0


In [11]:
# Create master product id
num_ints_for_product_id = 6
products_v2['merged_id'] = products_v2.apply(lambda x: utils.select_non_null_value(x['seqNo'], x['idproduct']), axis=1)
products_v2['master_product_id'] = products_v2.apply(lambda x: utils.create_product_id(x['merged_id'], num_ints_for_product_id, x['year_posted']), axis=1)

id_drop_cols = ['seqNo', 'idproduct', 'merged_id']
products_v2.drop(id_drop_cols, axis=1, inplace=True)

In [12]:
products_v2.head()

,product_name,category,description,shipping_options,seller_name,price,payment_method,ship_from,ship_to,year_posted,adj_price,master_product_id
0,503 - Forwarding failure (Privoxy@localhost),-,-,-,-,-,-,-,-,2016,NaN,16-000001
1,OG KUSH 1/2 OZ $125 AAAA+,Drugs \n\t\t41793,OG Kush makes up the genetic backbone of West ...,\n\n\n$15.99\t\t\t\t\t\nCANADA WIDE\n\n\n\n\n\...,GWLOVE,$125,Escrow Yes,CANADA,Worldwide Worldwide,2016,125.0,16-000002
2,Dream Market Login - Featured anonymous market...,-,-,-,-,-,-,-,-,2016,NaN,16-000003
3,Every drug available on request,Drugs \n\t\t41804,Every kind of medicine I can arrange . Send me...,-,Hectorstuff,$1,Escrow Yes,dutch,Worldwide Worldwide,2016,1.0,16-000004
4,"Debug, Learn how crack windows",Digital Goods \n\t\t29334,"Debug, Learn how crack windows",-,darkmarket,$1,Escrow Yes,Worldwide,Worldwide Worldwide,2016,1.0,16-000005


In [ ]:
# Seperate category name and ID.
products_v2['category_revised'] = products_v2['category'].str.replace(r'\n', '')
products_v2['category_revised'] = products_v2['category_revised'].str.replace('\t', '')
products_v2['category_name'] = products_v2['category_revised'].str.findall(r'[a-zA-Z ]')
products_v2['category_name'] = products_v2['category_name'].apply(lambda x: ''.join(x))
products_v2['category_name'] = products_v2['category_name'].apply(lambda x: utils.replace_empty_cell_with_nan(x))


products_v2['category_id'] = products_v2['category_revised'].str.findall(r'[\d+ ]')
products_v2['category_id'] = products_v2['category_id'].apply(lambda x: ''.join(x))
products_v2['category_id'] = products_v2['category_id'].apply(lambda x: utils.replace_empty_cell_with_nan(x))

products_v2.drop(['category', 'category_revised'], axis=1, inplace=True)

---
## Extract Info From Product Name

**Regex Examples:**
```
df['method_1'] = df['text'].str.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}')
df['method_2'] = df['text'].str.findall(r'(\d{1,2}) (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (\d{2,4})')
df['method_3'] = df['text'].str.findall(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.]* (\d{1,2})[,]* (\d{2,4})')
df['method_4'] = df['text'].str.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]* \d{4}')
df['method_5'] = df['text'].str.findall(r'\d{1,2}[/-]\d{4}')
df['method_6'] = df['text'].str.findall(r'\d{4}')
```    

In [ ]:
products_v2['prod_name_arr'] = products_v2['product_name'].apply(lambda x: utils.clean_and_split_text(x, r"[^a-zA-Z0-9$%.@'/_]"," "))

In [ ]:
products_v2.head()

---
## Extract Info From Description

In [ ]:
products_v2['desc_arr'] = products_v2['description'].apply(lambda x: utils.clean_and_split_text(x, r"[^a-zA-Z0-9$%.'/_]"," "))

In [ ]:
products_v2.iloc[50:58, :]

--- 
## Extract Info From "Shipping Options"

In [ ]:
products_v2['shipping_cleaned'] = products_v2['shipping_options'].apply(lambda x: utils.clean_and_split_text(x, r"[^a-zA-Z0-9$%.'/_]"," "))

In [ ]:
products_v2.iloc[15:20, :]

In [ ]:
products_v2['idproduct'].max()

In [ ]:
products_v2['seqNo'].max()

In [ ]:
products_v2.shape

### Ship From Formatting

In [ ]:
# NEED TO FINISH FUNCTION

def replace_uk_usa(text):
    try:
        text = re.sub(r'*united kingdom*', 'uk')
        text


def lowercase_and_split_string(text):
    '''
    
    
    '''
    
    if type(text) == str:
        text = text.lower()
        text = text.split(' ')

    return text

In [ ]:
products_v2['ship_from_cleaned'] = products_v2['ship_from'].apply(lambda x: lowercase_and_split_string(x))

In [ ]:
ship_from_arr = []
for obs in products_v2['ship_from_cleaned']:
    if type(obs) == list:
        for l in obs:
            if l in country_map:
                ship_from_arr.append(country_map[l])
            else:
                ship_from_arr.append(l)
ship_from_arr = pd.Series(ship_from_arr)

In [ ]:
print('Length - Original : ', products_v2.shape[0])
print('# of Unique Values - Original', len(products_v2['ship_from'].unique()))

print('Length - Revised: ', len(ship_from_arr))
print('# of Unique Value - Original: ', len(ship_from_arr.unique()))

In [ ]:
ship_from_arr.value_counts()[:10]

In [ ]:
products_v2.head()

In [ ]:
products_v2.iloc[100100:100110, :]

## EDA

In [ ]:
utils.stats_summary(products_v2['adj_price'])

In [ ]:
plt.figure(figsize=(25,8))
less_than_100 = products_v2[products_v2['adj_price'] < 100]['adj_price']
sns.distplot(less_than_100)
plt.title('Price Distribution of Products Less Than 100', fontsize= 20);
print('# - Goods Less Than $100: ', len(less_than_100))
print('% - Total Good Less Than $100: {:.1%}'.format(len(less_than_100) / products.shape[0]))

In [ ]:
fig, ax = plt.subplots(figsize=(25,12))
ax = sns.countplot(x='category_name', data=products_v2)
ax.set_title('Distribution of Product Type', fontsize=22)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.tight_layout();